In [1613]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)
library(ggplot2)
library(dummies)
library(stringr)
library(rms)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_score_finale.Rdata")
df <- df_score_finale
df <- subset(df,select= -c(degenzaPostTraOSP))

In [1614]:
names(df) <- c('Centre', 'Gender', 'Age', 'LHOSpretx', 'Status', 'VaricealBleed', 'Ascites', 'Tips',  
                'PVthrombosis', 'NutritStatus', 'Previoussurg', 'Ventilationpretx', 'Vasopressors', 
               'InfusionAlb', 'PaO2FiO2', 'PlatCount', 'Albumin', 'Duration', 'PRBCintraop', 'FFPintraop', 'Plateletsintraop',
               'Fibrinogen', 'CRIO', 'Crystalloids', 'PPS5', 'Norepinend', 'Lactates', 'ColdIschemia', 'ReperfusSy', 'PCshunt',
               'Cavaveinanasth', 'VVBypass', 'Biliaryanasth', 'Openabdomen', 'Donorgender', 'Donortype', 'Organtype' ,'DonorECMO', 
               'RegionalPerf', 'OrganBiopsy', 'Machineperf', 'DonorSite', 'DonorCauseDeath', 'DonorAge', 'DonorHeight', 'DonorSodiemia',
               'DonorGGT', 'MELDNa', 'Alcoholic', 'cirrhosisDrugs', 'NASH', 'Lateretx', 'DonorMaastricht', 'DonorSteatosis' ,
               'HypoNormoter', 'Nocomorbodities', 'Hypertension', 'SeverePsychosis', 'Periodstx', 'Nighttx', 'Admsite' ,
               'Encephalopathy', 'Cholestatic', 'Rare', 'Viral', 'HCC', 'Respcomorbidities', 'CVcomorbidities', 'Neurocomorbidities',
               'Renalcomorbidities', 'Diabetes', 'Anticoagulation', 'MELDExcept', 'deg_cat')

In [1001]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +42 secs: 

 confirmed 11 attributes: Albumin, Centre, Duration, Lactates, LHOSpretx and 6 more;

 rejected 39 attributes: Age, Alcoholic, Anticoagulation, Cholestatic, cirrhosisDrugs and 34 more;

 still have 23 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +50 secs: 

 confirmed 2 attributes: Ascites, PPS5;

 rejected 5 attributes: DonorGGT, Donortype, Organtype, Viral, VVBypas

 [1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Admsite"          "Encephalopathy"   "HCC"             


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [600]:
par(mar = c(13, 5, 2, 0.1))  
plot(boruta_output, cex.axis=1.3, las=2, xlab="", main="Variable Importance")

In [818]:
df_boruta$deg_cat <- df$deg_cat

In [1090]:
boruta_signif #variabili selezionate

[1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Admsite"          "Encephalopathy"   "HCC"

In [1615]:
df$deg_cat_bin = ifelse(df$deg_cat == 'NotFast',1,0)

df <- subset(df, select = -deg_cat)

In [1616]:
df <- df[df$PRBCintraop < 52,] # 4 big outliers
# df <- df[df$Duration < 1200,] #3 big outliers

In [1617]:
df <- subset(df, select = c('MELDNa','PRBCintraop','deg_cat_bin'))

In [1618]:
df$dataset <- sample(x=c("train","validate"),
size=nrow(df),
replace=TRUE,
prob=c(0.75,0.25))

## LOESS smoothing curves for numeric variables

In [1619]:
df.cont<- df[df$dataset=="train",
c('deg_cat_bin','MELDNa','PRBCintraop')]




ymark<-seq(0,1,0.1)

for(var in names(df.cont)[-1]){
xvar<-seq(min(df.cont[,var]),
max(df.cont[,var]),
length.out=10)
    
    
mypath <- file.path("C:/Users/abonini/Documents/score",
paste(paste("plot",var,sep = "_"),
"pdf", sep = "."))

pdf(file=mypath)

gg<-ggplot(df.cont, aes(x=df.cont[,var],y=deg_cat_bin))+
geom_jitter(size=1, alpha=0.2,height=0.05)+
stat_smooth(method="loess",colour="blue", size=1.5)+
xlab(var)+
ylab("Probability on length of hospital stay")+
theme_bw()+
geom_hline(yintercept=ymark,col="red")+
scale_y_continuous(breaks=ymark)+
geom_vline(xintercept=xvar,col="green",alpha=0.5)+
scale_x_continuous(breaks=round(xvar,2))
    
print(gg)
dev.off()
}

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'



## Choosing cut points and base reference

In [1620]:
MELDNacut<-c(10,17,30)
MELDNab<-"[2.05,10]"

PRBCintraopcut <-c(3,5,7)
PRBCintraopb<-"[0,3]"




cont.to.cat<-data.frame(id=c(1:nrow(df.cont)))

for (var in names(df.cont)[-1]) {
cat<-cut(df.cont[,var],
breaks=c(min(df.cont[,var]),
get(paste(var,"cut",sep="")),
max(df.cont[,var])),include.lowest= TRUE)
cat<-relevel(cat,ref=get(paste(var,"b",sep="")))
cont.to.cat<-cbind(cont.to.cat,cat)
}

## Calculating scores for each level

In [1621]:
df.cont.to.cat<-cont.to.cat[,-1] 

names(df.cont.to.cat)<-names(df.cont)[-1]

df.final<-cbind(df.cont.to.cat,

deg_cat_bin=df.cont$deg_cat_bin)

mod<-glm(deg_cat_bin~.,
df.final, family="binomial")
score<-round(coef(mod)[-1] * 10)


score.cont<-score

In [1623]:
summary(mod)


Call:
glm(formula = deg_cat_bin ~ ., family = "binomial", data = df.final)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1227  -0.9907   0.5053   0.9772   1.3763  

Coefficients:
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -0.4564     0.1396  -3.270 0.001075 ** 
MELDNa(10,17]       0.6802     0.1802   3.774 0.000161 ***
MELDNa(17,30]       1.4785     0.2106   7.020 2.22e-12 ***
MELDNa(30,48.3]     2.3309     0.4331   5.382 7.36e-08 ***
PRBCintraop(3,5]    0.2674     0.2034   1.314 0.188729    
PRBCintraop(5,7]    0.8591     0.2952   2.910 0.003609 ** 
PRBCintraop(7,44]   0.9719     0.2662   3.650 0.000262 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1220.2  on 944  degrees of freedom
Residual deviance: 1070.7  on 938  degrees of freedom
AIC: 1084.7

Number of Fisher Scoring iterations: 5


In [1624]:
score

MELDNa(10,17]     MELDNa(17,30]   MELDNa(30,48.3]  PRBCintraop(3,5] 
                7                15                23                 3 
 PRBCintraop(5,7] PRBCintraop(7,44] 
                9                10

In [1625]:
head(df.final)

MELDNa,PRBCintraop,deg_cat_bin
"(10,17]","(3,5]",0
"(17,30]","(7,44]",1
"[2.05,10]","[0,3]",0
"[2.05,10]","[0,3]",0
"(30,48.3]","(5,7]",1
"[2.05,10]","[0,3]",1


## Calculating scores for numeric variables

In [1626]:
var.cont<-as.character(1)

for(var in names(score.cont)){
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
var.cont<-c(var.cont,var.red)
}

var.cont<-unique(var.cont)[-1]

for(var in var.cont){
df[,paste(var,"points",sep=".")]<-as.numeric(NA)
}

for (var in names(score.cont)){
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
var.low<-as.numeric(str_extract(var,'(?<=(\\(|\\[))[0-9]+\\.*[0-9]*(?=\\,)'))
var.upper<-as.numeric(str_extract(var,'(?<=\\,)[0-9]+\\.*[0-9]*(?=\\])'))
df[,paste(var.red,"points",sep=".")]<-ifelse(
df[,var.red]<=var.upper&df[,var.red]>=var.low,
score[var],df[,paste(var.red,"points",sep=".")])
}

for(var in var.cont){
df[,paste(var,"points",sep=".")]<-ifelse(
is.na(df[,paste(var,"points",sep=".")]),
0,df[,paste(var,"points",sep=".")])
}

# Calculate scores for factor variables and take a sum

var.cat<-names(df.cat)

for(var in var.cat){
    df[,paste(var,"points",sep=".")]<-as.numeric(NA)
}

for (var in var.cat){
score.var<-score.cat[grep(var,names(score.cat))]
        names(score.var)<-sub(var,"",names(score.var))
    for(i in 1:(length(levels(df[,var]))-1)){
        df[,paste(var,"points",
        sep=".")]<-ifelse(
        df[,var]==names(score.var)[i]&
        is.na(df[,paste(var,"points",sep=".")]),
        score.var[i],
    df[,paste(var,"points",sep=".")])}
    }

for(var in var.cat){
    df[,paste(var,"points",sep=".")]<-ifelse(
    is.na(df[,paste(var,"points",sep=".")]),
    0,df[,paste(var,"points",sep=".")])}

### Non uso categoriche

In [1627]:
df$score<-rowSums(df[,grepl("\\.+points",names(df))])

In [1628]:
head(df)

,MELDNa,PRBCintraop,deg_cat_bin,dataset,MELDNa.points,PRBCintraop.points,score
NA.575,10.26,5,0,train,7,9,16
NA.577,26.68,44,1,train,15,10,25
NA.682,4.44,0,0,train,0,0,0
NA.687,25.78,14,1,validate,15,10,25
NA.692,4.14,0,0,train,0,0,0
NA.693,7.29,1,0,validate,0,0,0


In [1629]:
score

MELDNa(10,17]     MELDNa(17,30]   MELDNa(30,48.3]  PRBCintraop(3,5] 
                7                15                23                 3 
 PRBCintraop(5,7] PRBCintraop(7,44] 
                9                10

## Converting the score to probability of mortality and comparing it to observed number of deaths

In [1630]:
glmod<-glm(deg_cat_bin~score,
df[df$dataset=="train",],
family="binomial")

newx<-seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score))

prd<-predict(glmod,
newdata=data.frame(score=newx),
type="response",
se.fit=T)

count<-as.matrix(table(cut(df[df$dataset=="train",]$score,
breaks=seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score)),
include.lowest = T),
df[df$dataset=="train",]$deg_cat_bin))


In [1631]:
summary(glmod)


Call:
glm(formula = deg_cat_bin ~ score, family = "binomial", data = df[df$dataset == 
    "train", ])

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2872  -0.9733   0.5343   0.8176   1.3963  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.501152   0.117662  -4.259 2.05e-05 ***
score        0.095025   0.008644  10.993  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1220.2  on 944  degrees of freedom
Residual deviance: 1071.8  on 943  degrees of freedom
AIC: 1075.8

Number of Fisher Scoring iterations: 4


## Visualization of the relationship between scores and probability of outcome events

In [1635]:
par(mar=c(5,4,4,5)+.1)

barplot(t(count),
main="Scores versus probability on length of hospital stay",
xlab="Scores",
ylab="Observed number of patients",
space=0,
col=c("yellow","lightblue"))

legend("topleft",fill=c("yellow","lightblue",NA),
lty = c(NA,NA,1),lwd=c(NA,NA,2),
legend=c("fast","NotFast",
"Predicted Prob."),
col=c("black"),
border = c("black","black",NA))

par(new=TRUE)

plot(prd$fit~newx,
type="l",col="black",
lwd=2,xaxt="n",yaxt="n",
xlab="",ylab="")

polygon(c(rev(newx), newx),
c(rev(prd$fit+1.96*prd$se.fit),
prd$fit-1.96*prd$se.fit),
col = adjustcolor('grey80',alpha=0.5),
border = NA)

lines(newx, prd$fit+1.96*prd$se.fit,
lty = 'dashed', col = 'red')

lines(newx, prd$fit-1.96*prd$se.fit,
lty = 'dashed', col = 'red')

axis(4)

mtext("Predicted probability length of hospital stay",
side=4,line=3)


## Validation of the score

In [1636]:
ddist <- datadist(df)

options(datadist='ddist')

f.score<-lrm(deg_cat_bin~score,
df[df$dataset=="train",],
x=TRUE,y=TRUE)

phat.score<-predict(f.score,
df[df$dataset=="validate",],
type="fitted")

v.score<-val.prob(phat.score,
df[df$dataset=="validate",]$deg_cat_bin,
m=20)


# Prediction

In [1637]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

In [1638]:
df$deg_cat_bin <- ifelse(df$deg_cat_bin == 1,"NotFast","fast")

In [1639]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [1640]:
df_train <- df[df$dataset == "train",]

In [1641]:
df_train <- SmoteClassif(deg_cat_bin ~ score, df_train, k = 27, dist = "Manhattan")

#view distribution of response variable in new dataset
table(df_train$deg_cat_bin)


Warning message in Smote.exsClassif(dat[which(dat[, ncol(dat)] == li[[1]][ove[i]]), :
"si è prodotto un NA per coercizione"
Warning message in lapply(X = X, FUN = FUN, ...):
"si è prodotto un NA per coercizione"


ERROR: Error in neighbours(tgt, dat, dist, p, k): Can not compute Manhattan distance with nominal attributes!


In [1642]:
df$score

[1] 16 25  0 25  0  0 32 32  0 24 18  0  0  7 23 25  7 15  0  7 24  3  0 17
  [25]  7 25 24  0 25 16  0  0  7  0  3 16  7  7 24 24 25 25 17  7 15 10 25 16
  [49] 18  7 16 16 33  7  7  9 10 25  0  0 10  0 32  0 32  0 26 33  3  9 15 10
  [73] 15 16  9  0 33  0 16  7 10  0 33  7 24 25  0 16  7  7  3  0 18 17  0 10
  [97]  3  0  7  7  0 17  0 18  0  7 15 10 10  7 24 32  7 26  0  0  0  7  7  7
 [121] 15  0 15  7 23  7  7  7  0  7 32 24  7 18  0  0  0  0  7 24  0 15  7 33
 [145] 33 25 32 18  0  7 18  0  7  7 15 15 26 18 15 15 17 15 24 25 25 10  7  0
 [169] 24 18 32 18 15 15 26 15 18 33  3 15  0 18  3 10  7 18 32  7 15  7 10  0
 [193]  7 26 18 24 26 33  7  0 10 24 25  7 24  0 25 17 33 25  9 15 10  0 17 17
 [217]  0  0 24 25 17  0  7 25 33  0  3  9 24 33 24 18 25  7  0 17 18 25  7 10
 [241] 10  0 33  0  3 25  0  7  7  7  9 25  0  0 25  7 33  0 25  3  7  3  7 17
 [265]  0 17 25  0 15  7  0  0  0  9 15 18 33 33 16 16 33  9  0  0  0 25  0 10
 [289] 17 25 25 18 33  7 18  7 16 25 24 24 25 26 17 25 25 10 10  0 18 33  7 25
 [313] 15 24 26  0 33 25 24 24 25  0  7 17 24 25 17 17  0 15 17  0 33  0 16 15
 [337]  7  7 10  7 25 10 24 25 16 33  7  0 25  0 24 24  7 16  7 15 33  0  0  0
 [361] 25  0  7 25  0 10 33 25  0  7  0 25 25  0  0 15  0 24 18 17 25  7  7  0
 [385]  0  7 24  0 25 17 10  0 25 25 17 24 16  7  0  0 17 26 18  7  7 17  0 32
 [409] 10 10 15  0  0 33 18  3  7  0 10 26 18 16 18  0  7  9 16  0  0  9 25 24
 [433] 10 18 25  0  0 15  0  0 10 15  7 23 25  0 17 18 16 10 15 18 17 10  0  7
 [457]  7  0 17 15 18 18  7 33 10 25 18 10 10  0 18 25  0 16  7 33 15 33  7 24
 [481] 23  0 24  7 18 18 18 15 15 25 24  7 24 23 10 32 15 15 18 15  0 33 18  0
 [505]  7 15 18 10 18  7 18  0  7 23 24 24  7 15 18  7 24 10  7 18 32 24 25  7
 [529] 18 26 24 18 26 23 18  7 18 33  7  7 10  7 10  7  3 16 18  7 10 15 25 32
 [553]  7 33 24 16 25 33 25 17 25 10 26  7 23  7 10  0 16  0 25 10 25  7  9 25
 [577] 15 15 17 16  0 33 17 25 17  7 10 25 18 10  3 25 25 10  0  7 16 33 15  0
 [601] 24 18  7 15  7  0  0 24 33 25  0 26 24 18  0  0  0  7 25 25  7 15 17  7
 [625] 10  0  0 18 25 24 25 33 18 25  7 25 17 24 25 18  9  7 25 16 10  0 25 10
 [649] 24 10 10 25 26 16 18 26  0  3 25 15 18  7  7 24 25  0 24  7 25 10  0 25
 [673] 10  0  0  7 24 15 16 18 25 10 10 25 10 16 10  0 17 25 16 15  0 16  7  7
 [697]  0  0  7  7  0 24 17  7 32  7 25 17 25 17 15  3  3 25 25  0  7 18  0  7
 [721]  7  7  0  0 10 24 25  9 26 16  0  0 10 24 18 32  0 18  7 15  7  7  7 33
 [745]  7  0 16 25  7  0  9 15 25 25  7 26 25 26  7  7  7 25  0  7 16 25  0 15
 [769] 15 33  7 25 25 25 25 24 10 10 15  7 18 24  0 15 26 17  7  0  7  0  0 32
 [793] 18 24 17 15  0 18 10 17 18 10  0  0  0  0  7  3 10 24 10 25  7 15  9  7
 [817]  7  0  0 10  7 33 10  0  0 33  7  7 10  9  7  0  0  0  0 17 15  7  0  7
 [841]  7 17 25 10  0  0  7  7  7  0  7  7 25 25  7 25  7 18  0  7  0  7 17  0
 [865]  7  7 24 24 15  0 17 17  9 17  0 26  7 15 15 10  7  7 33  7  7 15 18 18
 [889]  0  7  7 15 15 15 24 25  7  7 25 16 23  0 18 18 18 24 10 16  7  0 10  9
 [913] 33 18  0  0 18  3 15  7  0  0 15  0 24 18  0 33  0 24 15  7  7  0  7 33
 [937] 15 18 24 10 18  0 24  3  7 33  7  0 25  0  0 26  0  7 15 18 10 15  7  7
 [961]  0 15  7  7 10 10  7 33  0  7 10 10  7 33 15  7 15 10 15 10  7  7 18 24
 [985] 24 17 17  7 15 25 16 10 25  7  0  0 18 24 25 18 10 25 16 10 10 15 25 25
[1009]  9 25  0 10 18 10 33 17 10 25  0 25 33 10 17  0  7 25 15  3 25 17  0  9
[1033] 33 25  3  0 18 16 25  0  0 33 15  7 18  7 33 15 18 25  0  0 33  7 33 18
[1057] 17 25 18  7 10 15  7  0  0  0 15 24 25 33  0  0 16  3 24 16  7 33 17 18
[1081] 33 16 25 25  7  7  0  9 15 10 15 15 33 10 18 16 25 24  0 25  7  7 33 24
[1105]  7 15 26  9  7  7  7 16 18 17  0 18  0  7  0 33 18 18  7 10  3  7 25 15
[1129] 33 18  7 17  7  7  0  0  7 32 23  7  3  0  7 25 18 23 25 10 24  0  7  0
[1153] 18 25  7  7  7  0 17 25 24 18  7  7 18 15 25 25  7 16  0 18  3  3 15 15
[1177] 15 33 25 26 18  7 15 18  0 15 15  7 25 33 15 33 10 15 25 15  0 16 10 18
[1201]  7  7 15  0  0 18  7  7 25 32 18  3 18 15  0 1

In [1643]:
table(df_train$deg_cat_bin)


   fast NotFast 
    328     617 

In [1644]:
#Regularized Logistic Regression
model_regLogistic <- train(deg_cat_bin ~ score , data = df_train,
               method = "regLogistic",
               trControl = ctrl)    # metric = "..." ciò che voglio massimizzare

+ Fold01.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
- Fold01.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
+ Fold01.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
- Fold01.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
+ Fold01.Rep1: cost=0.018786, loss=L1, epsilon=1 
- Fold01.Rep1: cost=0.018786, loss=L1, epsilon=1 
+ Fold02.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
- Fold02.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
+ Fold02.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
- Fold02.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
+ Fold02.Rep1: cost=0.018786, loss=L1, epsilon=1 
- Fold02.Rep1: cost=0.018786, loss=L1, epsilon=1 
+ Fold03.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
- Fold03.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
+ Fold03.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
- Fold03.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
+ Fold03.Rep1: cost=0.018786, loss=L1, epsilon=1 
- Fold03.Rep1: cost=0.018786, loss=L1, epsilon=1 
+ Fold04.Rep1: c

In [1645]:
print(model_regLogistic)
plot(model_regLogistic)

Regularized Logistic Regression 

945 samples
  1 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 851, 851, 850, 851, 851, 851, ... 
Resampling results across tuning parameters:

  cost         loss       Accuracy   Kappa     
  0.001162181  L2_primal  0.6529115  0.00000000
  0.002923303  L2_primal  0.6529115  0.00000000
  0.018785553  L1         0.6561030  0.02239956

Tuning parameter 'epsilon' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were cost = 0.01878555, loss = L1
 and epsilon = 1.


In [1646]:
df_test <- df[df$dataset == "validate",]
test <- df_test[,"deg_cat_bin"]
test <- as.factor(test)

In [1647]:
pred_regLogistic <- predict(model_regLogistic,df_test)  
pred_regLogistic.prob <- predict(model_regLogistic,df_test, type="prob")


result <- confusionMatrix(test,  pred_regLogistic) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast       0      93
   NotFast    0     219
                                          
               Accuracy : 0.7019          
                 95% CI : (0.6478, 0.7521)
    No Information Rate : 1               
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity :     NA          
            Specificity : 0.7019          
         Pos Pred Value :     NA          
         Neg Pred Value :     NA          
             Prevalence : 0.0000          
         Detection Rate : 0.0000          
   Detection Prevalence : 0.2981          
      Balanced Accuracy :     NA          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
                  NA            0.7019231                   NA 
      Neg Pred Value            Precision               Recall 
                  NA            0.0000000                   NA 
                  F1           Prevalence       Detection Rate 
                  NA            0.0000000            0.0000000 
Detection Prevalence    Balanced Accuracy 
           0.2980769                   NA